In [1]:
# MTA turnstile data for all stations
# week: November 10-16, 2018
# extracting: M-F, AM, PM
# average riders per stop for AM and PM

In [2]:
# import Dependencies
import pandas as pd
import numpy as np

In [3]:
# data from MTA: http://web.mta.info/developers/turnstile.html
csv_path = "turnstile_181117.csv"
ts_11_2018_all = pd.read_csv(csv_path, encoding="utf-8")

In [ ]:
# format date and time for Pandas
ts_11_2018_all["DATE"] = pd.to_datetime(ts_11_2018_all["DATE"])
ts_11_2018_all["TIME"] = pd.to_datetime(ts_11_2018_all["TIME"])

In [ ]:
# add 'day of week' column - prep to remove Saturdy and Sunday
ts_11_2018_all["WEEKDAY"] = ts_11_2018_all["DATE"].dt.day_name()

In [ ]:
# create new dataframe, as to not completely lose weekend data
ts_11_2018 = ts_11_2018_all

In [ ]:
# use .loc to remove Saturday annd Sunday
ts_11_2018 = ts_11_2018.loc[(ts_11_2018["WEEKDAY"]!= "Saturday") & (ts_11_2018["WEEKDAY"]!= "Sunday"), : ]
ts_11_2018.head()

In [115]:
# check for missing station data
# ts_11_2018[ts_11_2018['STATION'] == '42 ST-PORT AUTH']

In [7]:
# check column titles because there is a problem with EXIT
ts_11_2018.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               ',
       'WEEKDAY'],
      dtype='object')

In [8]:
# remane columns: entries / exits, as these are cummulative
ts_11_2018 = ts_11_2018.rename(columns={'ENTRIES': 'TOT. ENTRIES'})
ts_11_2018 = ts_11_2018.rename(columns={'EXITS                                                               '
                                                      : 'TOT. EXITS'})
ts_11_2018.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,TOT. ENTRIES,TOT. EXITS,WEEKDAY
12,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-23 03:00:00,REGULAR,6831427,2316424,Monday
13,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-23 07:00:00,REGULAR,6831438,2316474,Monday
14,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-23 11:00:00,REGULAR,6831570,2316713,Monday
15,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-23 15:00:00,REGULAR,6831812,2316785,Monday
16,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-23 19:00:00,REGULAR,6832592,2316857,Monday


In [9]:
# enter/exit are cummulative, need to subtract from previous
# copying ENTERS, shifting down a row, then substracting
# using abs to give us a positive number
# create new column to hold entries
ts_11_2018['ENTERS'] = abs(ts_11_2018['TOT. ENTRIES'].shift(1) - ts_11_2018['TOT. ENTRIES'])

In [10]:
# need to repeat steps for exits
ts_11_2018['EXITS'] = abs(ts_11_2018['TOT. EXITS'].shift(1) - ts_11_2018['TOT. EXITS'])
ts_11_2018.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,TOT. ENTRIES,TOT. EXITS,WEEKDAY,ENTERS,EXITS
12,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-23 03:00:00,REGULAR,6831427,2316424,Monday,NaN,NaN
13,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-23 07:00:00,REGULAR,6831438,2316474,Monday,11.0,50.0
14,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-23 11:00:00,REGULAR,6831570,2316713,Monday,132.0,239.0
15,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-23 15:00:00,REGULAR,6831812,2316785,Monday,242.0,72.0
16,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-23 19:00:00,REGULAR,6832592,2316857,Monday,780.0,72.0


In [11]:
# new data frame = focus on morning commute: DAMN YOU, MTA!
ts_11_2018_am = ts_11_2018

In [13]:
# extract AM data
ts_11_2018_am = ts_11_2018_am.loc[(ts_11_2018_am["TIME"]=="05:00:00") | (ts_11_2018_am["TIME"]=="09:00:00") |
                                  (ts_11_2018_am["TIME"]=="05:15:28") | (ts_11_2018_am["TIME"]=="05:30:00") |
                                  (ts_11_2018_am["TIME"]=="06:38:21") | (ts_11_2018_am["TIME"]=="07:00:00") | 
                                  (ts_11_2018_am["TIME"]=="07:22:00") | (ts_11_2018_am["TIME"]=="07:25:28") |
                                  (ts_11_2018_am["TIME"]=="07:30:00") | (ts_11_2018_am["TIME"]=="07:38:52") |
                                  (ts_11_2018_am["TIME"]=="07:50:33") | (ts_11_2018_am["TIME"]=="07:52:47") |
                                  (ts_11_2018_am["TIME"]=="07:53:17") | (ts_11_2018_am["TIME"]=="08:00:00") |
                                  (ts_11_2018_am["TIME"]=="08:02:06") | (ts_11_2018_am["TIME"]=="08:02:10") |
                                  (ts_11_2018_am["TIME"]=="08:02:16") | (ts_11_2018_am["TIME"]=="08:02:19") |
                                  (ts_11_2018_am["TIME"]=="08:02:23") | (ts_11_2018_am["TIME"]=="08:02:25") |
                                  (ts_11_2018_am["TIME"]=="08:02:28") | (ts_11_2018_am["TIME"]=="08:02:35") |
                                  (ts_11_2018_am["TIME"]=="08:02:39") | (ts_11_2018_am["TIME"]=="08:02:43") |
                                  (ts_11_2018_am["TIME"]=="08:02:46") | (ts_11_2018_am["TIME"]=="08:02:51") |
                                  (ts_11_2018_am["TIME"]=="08:02:54") | (ts_11_2018_am["TIME"]=="08:02:59") |
                                  (ts_11_2018_am["TIME"]=="08:03:06") | (ts_11_2018_am["TIME"]=="08:03:15") |
                                  (ts_11_2018_am["TIME"]=="08:03:21") | (ts_11_2018_am["TIME"]=="06:00:00") |
                                  (ts_11_2018_am["TIME"]=="08:03:27") | (ts_11_2018_am["TIME"]=="08:03:32") |
                                  (ts_11_2018_am["TIME"]=="08:03:35") | (ts_11_2018_am["TIME"]=="08:03:38") |
                                  (ts_11_2018_am["TIME"]=="08:08:31") | (ts_11_2018_am["TIME"]=="08:21:07") |
                                  (ts_11_2018_am["TIME"]=="08:22:45") | (ts_11_2018_am["TIME"]=="08:23:47") |
                                  (ts_11_2018_am["TIME"]=="08:25:07") | (ts_11_2018_am["TIME"]=="08:26:47") |
                                  (ts_11_2018_am["TIME"]=="08:28:35") | (ts_11_2018_am["TIME"]=="08:28:39") |
                                  (ts_11_2018_am["TIME"]=="08:28:44") | (ts_11_2018_am["TIME"]=="08:29:08") |
                                  (ts_11_2018_am["TIME"]=="08:30:44") | (ts_11_2018_am["TIME"]=="08:32:04") |
                                  (ts_11_2018_am["TIME"]=="08:33:24") | (ts_11_2018_am["TIME"]=="08:56:08") |
                                  (ts_11_2018_am["TIME"]=="09:00:00") | (ts_11_2018_am["TIME"]=="09:01:02") |
                                  (ts_11_2018_am["TIME"]=="09:10:42") | (ts_11_2018_am["TIME"]=="09:11:13") |
                                  (ts_11_2018_am["TIME"]=="09:11:16") | (ts_11_2018_am["TIME"]=="09:11:21") |
                                  (ts_11_2018_am["TIME"]=="09:15:28") | (ts_11_2018_am["TIME"]=="09:17:12") |
                                  (ts_11_2018_am["TIME"]=="09:17:16") | (ts_11_2018_am["TIME"]=="09:17:21") |
                                  (ts_11_2018_am["TIME"]=="09:17:40") | (ts_11_2018_am["TIME"]=="09:17:46") |
                                  (ts_11_2018_am["TIME"]=="09:17:54") | (ts_11_2018_am["TIME"]=="09:18:08") |
                                  (ts_11_2018_am["TIME"]=="09:18:13") | (ts_11_2018_am["TIME"]=="09:18:17") |
                                  (ts_11_2018_am["TIME"]=="09:18:21") | (ts_11_2018_am["TIME"]=="09:18:24") |
                                  (ts_11_2018_am["TIME"]=="09:18:27") | (ts_11_2018_am["TIME"]=="09:18:31") |
                                  (ts_11_2018_am["TIME"]=="09:18:35") | (ts_11_2018_am["TIME"]=="09:18:38") |
                                  (ts_11_2018_am["TIME"]=="09:18:42") | (ts_11_2018_am["TIME"]=="09:18:46") |
                                  (ts_11_2018_am["TIME"]=="09:18:48") | (ts_11_2018_am["TIME"]=="09:19:00") |
                                  (ts_11_2018_am["TIME"]=="09:19:03") | (ts_11_2018_am["TIME"]=="09:19:15") |
                                  (ts_11_2018_am["TIME"]=="09:19:07") | (ts_11_2018_am["TIME"]=="09:19:11") |
                                  (ts_11_2018_am["TIME"]=="09:19:12") | (ts_11_2018_am["TIME"]=="09:19:17") |
                                  (ts_11_2018_am["TIME"]=="09:19:21") | (ts_11_2018_am["TIME"]=="09:19:22") |
                                  (ts_11_2018_am["TIME"]=="09:19:27") | (ts_11_2018_am["TIME"]=="09:19:31") |
                                  (ts_11_2018_am["TIME"]=="09:19:38") | (ts_11_2018_am["TIME"]=="09:19:39") |
                                  (ts_11_2018_am["TIME"]=="09:19:43") | (ts_11_2018_am["TIME"]=="09:19:45") |
                                  (ts_11_2018_am["TIME"]=="09:19:48") | (ts_11_2018_am["TIME"]=="09:19:56") |
                                  (ts_11_2018_am["TIME"]=="09:20:00") | (ts_11_2018_am["TIME"]=="09:20:15") |
                                  (ts_11_2018_am["TIME"]=="09:20:31") | (ts_11_2018_am["TIME"]=="09:20:34") |
                                  (ts_11_2018_am["TIME"]=="09:20:37") | (ts_11_2018_am["TIME"]=="09:20:51") |
                                  (ts_11_2018_am["TIME"]=="09:21:03") | (ts_11_2018_am["TIME"]=="09:21:06") |
                                  (ts_11_2018_am["TIME"]=="09:21:11") | (ts_11_2018_am["TIME"]=="09:21:13") |
                                  (ts_11_2018_am["TIME"]=="09:21:14") | (ts_11_2018_am["TIME"]=="09:21:17") |                               
                                  (ts_11_2018_am["TIME"]=="09:21:23") | (ts_11_2018_am["TIME"]=="09:21:26") |                               
                                  (ts_11_2018_am["TIME"]=="09:21:29") | (ts_11_2018_am["TIME"]=="09:21:32") |                               
                                  (ts_11_2018_am["TIME"]=="09:21:37") | (ts_11_2018_am["TIME"]=="09:21:40") |                               
                                  (ts_11_2018_am["TIME"]=="09:21:46") | (ts_11_2018_am["TIME"]=="09:21:53") |                               
                                  (ts_11_2018_am["TIME"]=="09:22:02") | (ts_11_2018_am["TIME"]=="09:22:05") |                               
                                  (ts_11_2018_am["TIME"]=="09:22:12") | (ts_11_2018_am["TIME"]=="09:23:10") |                               
                                  (ts_11_2018_am["TIME"]=="09:29:18") | (ts_11_2018_am["TIME"]=="09:30:00") , :]
ts_11_2018_am.head()

,STATION,LINENAME,DATE,TIME,WEEKDAY,ENTERS,EXITS
13,59 ST,NQR456W,2018-11-12,2019-08-23 07:00:00,Monday,11.0,50.0
19,59 ST,NQR456W,2018-11-13,2019-08-23 07:00:00,Tuesday,16.0,46.0
25,59 ST,NQR456W,2018-11-14,2019-08-23 07:00:00,Wednesday,18.0,60.0
39,59 ST,NQR456W,2018-11-12,2019-08-23 07:00:00,Monday,11.0,19.0
45,59 ST,NQR456W,2018-11-13,2019-08-23 07:00:00,Tuesday,19.0,18.0


In [14]:
# clean up data frame
ts_11_2018_am = ts_11_2018_am.drop(['C/A','UNIT','SCP','DIVISION','DESC','TOT. ENTRIES','TOT. EXITS'], axis=1)
ts_11_2018_am.head()

KeyError: "['C/A' 'UNIT' 'SCP' 'DIVISION' 'DESC' 'TOT. ENTRIES' 'TOT. EXITS'] not found in axis"

In [22]:
# groupby station name and get total (not average)
ts_11_2018_am_tot = ts_11_2018_am.groupby(['STATION','LINENAME']).sum()

In [26]:
# change columns name; AM ENTERS / AM EXITS
# remane AM columns: entries / exits
ts_11_2018_am_tot = ts_11_2018_am_tot.rename(columns={'ENTERS': 'AM_ENTERS'})
ts_11_2018_am_tot = ts_11_2018_am_tot.rename(columns={'EXITS' : 'AM_EXITS'})
ts_11_2018_am_tot.head()

AM_ENTERS  AM_EXITS
STATION       LINENAME                     
1 AV          L            2173.0    5427.0
103 ST        1            7888.0    1943.0
              6            8439.0    7932.0
              BC           4581.0    1183.0
103 ST-CORONA 7           22039.0    1903.0

In [17]:
# check total number of stations we have
ts_11_2018_am_tot.count()

AM_ENTERS    469
AM_EXITS     469
dtype: int64

In [98]:
# save as csv
ts_11_2018_am_tot.to_csv('ts_11_2018_am_tot.csv')

In [99]:
# Repeat for PM commute
# new data frame = focus on evening commute: 3:00pm AND 4:00pm AND 5:00pm AND 5:30pm AND 3:22pm
ts_11_2018_pm = ts_11_2018

In [100]:
# look at only 3pm AND 4pm data
ts_11_2018_pm = ts_11_2018_pm.loc[(ts_11_2018_pm["TIME"]=="15:00:00") | (ts_11_2018_pm["TIME"]=="15:22:00") |
                                  (ts_11_2018_pm["TIME"]=="15:30:00") | (ts_11_2018_pm["TIME"]=="15:47:18") |
                                  (ts_11_2018_pm["TIME"]=="16:00:00") | (ts_11_2018_pm["TIME"]=="16:13:51") |
                                  (ts_11_2018_pm["TIME"]=="16:34:12") | (ts_11_2018_pm["TIME"]=="16:34:13") |
                                  (ts_11_2018_pm["TIME"]=="17:00:00") | (ts_11_2018_pm["TIME"]=="17:15:28") |
                                  (ts_11_2018_pm["TIME"]=="17:30:00") | (ts_11_2018_pm["TIME"]=="18:00:00") |
                                  (ts_11_2018_pm["TIME"]=="18:27:48") | (ts_11_2018_pm["TIME"]=="18:27:49") |
                                  (ts_11_2018_pm["TIME"]=="19:00:00"), :]

In [101]:
# clean up data frame
ts_11_2018_pm = ts_11_2018_pm.drop(['C/A','UNIT','SCP','DIVISION','DESC','TOT. ENTRIES','TOT. EXITS'], axis=1)

In [102]:
# groupby station name and get total (not average)
ts_11_2018_pm_tot = ts_11_2018_pm.groupby(['STATION','LINENAME']).sum()

In [103]:
# change columns name; PM ENTERS / PM EXITS
ts_11_2018_pm_tot = ts_11_2018_pm_tot.rename(columns={'ENTERS': 'PM_ENTERS'})
ts_11_2018_pm_tot = ts_11_2018_pm_tot.rename(columns={'EXITS' : 'PM_EXITS'})
ts_11_2018_pm_tot.head()

PM_ENTERS  PM_EXITS
STATION       LINENAME                     
1 AV          L           51881.0   51545.0
103 ST        1           13537.0    6671.0
              6           16862.0   12606.0
              BC           4818.0    3202.0
103 ST-CORONA 7           30974.0   40746.0

In [104]:
# save as csv
ts_11_2018_pm_tot.to_csv('ts_11_2018_pm_tot.csv')

In [105]:
# merge AM and PM into one dataframe
commuters_11_2018 = pd.merge(ts_11_2018_am_tot, ts_11_2018_pm_tot, on=["STATION", "LINENAME"], how="outer")
commuters_11_2018.head()

AM_ENTERS  AM_EXITS  PM_ENTERS  PM_EXITS
STATION       LINENAME                                          
1 AV          L            2173.0    5427.0    51881.0   51545.0
103 ST        1            7888.0    1943.0    13537.0    6671.0
              6            8439.0    7932.0    16862.0   12606.0
              BC           4581.0    1183.0     4818.0    3202.0
103 ST-CORONA 7           22039.0    1903.0    30974.0   40746.0

In [106]:
# save as csv
commuters_11_2018.to_csv('commuters_11_2018.csv')